#### Antiga forma de criar requests

Os requests são tuplas de ```(app, alice, bob, prioridade)```. Trocado por um objeto Request porque facilitava a atualização das chaves obtidas. Caminho antigo: ```components/qkd/app.py```

In [ ]:
# Simulação de QKD

def generate_qkd_request(rede, num_requests, avaliable_apps, diff_nodes=5):
        """
        Gera uma lista de requisições aleatórias de QKD.

        Args:
            num_requests (int): Número de requisições.
            diff_nodes (int): Número entre os nós. Defauts to 5.

        Returns:
            requests (list): Lista com requisições.
        """
        requests = []
        
        for i in range(num_requests):
            alice, bob = rede.random_alice_bob(diff_nodes)
            priority = random.randint(1, 5)
            app = random.choice(avaliable_apps)
            requests.append((alice, bob, app, priority))
        
        return requests
    
def run_simulations(rede, controlador, n_simulacoes, n_requests, avaliable_apps, routes_calculation_type='shortest'):
    """
    Roda as simulações para os protocolos BB84, E91 e B92.

    Args:
        rede (Network): Rede.
        controlador (Controller): Controlador.
        n_simulacoes (int): Número de simulações.
        n_requests (int): Número de requisições.
        routes_calculation_type (str): Tipo de roteamento. Defaults to 'shortest'.
    
    Returns:
        taxas_sucesso_chaves_geral (list): Lista com as taxas de sucesso das chaves para cada simulação.
        vazao (list): Lista com a vazão para cada simulação.
    """
    
    taxas_sucesso_chaves_geral = []
    vazao = []

    for simulacao in range(n_simulacoes):
        print("Simulação: ", simulacao)
        taxas_sucesso_chaves_e91 = []
        taxas_sucesso_chaves_bb84 = []
        taxas_sucesso_chaves_b92 = []
        
        requests = generate_qkd_request(rede, n_requests, avaliable_apps)
        print("Requests: ", requests)
        resultados_simulacao = controlador.send_requests(requests, routes_calculation_type)

        for indice_execucao in resultados_simulacao:
            resultado_individual_simulacao = resultados_simulacao[indice_execucao]
            sucesso_chave = resultado_individual_simulacao['key sucess']

            if resultado_individual_simulacao['app'] == 'BB84':
                taxas_sucesso_chaves_bb84.append(sucesso_chave)
                
            elif resultado_individual_simulacao['app'] == 'E91':
                taxas_sucesso_chaves_e91.append(sucesso_chave)
                
            elif resultado_individual_simulacao['app'] == 'B92':
                taxas_sucesso_chaves_b92.append(sucesso_chave)

        # Salvando o sucesso nas chaves geral da simulação
        lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
        taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
        
        # Calculando a vazão
        n_execucoes = len(resultados_simulacao)
        vazao.append(n_requests / n_execucoes)

    return taxas_sucesso_chaves_geral, vazao